In [5]:
#importing stuff we need later
import pandas as pd 
import csv

#taking the names of datasets we need to mesh - IMPORTANT: Datasets should be ordered by year and by NUTS code (this has to be done in the .csv file by selecting everything and order it by TIME first and CODE second)
name_1='USEFUL DATASETS/Meshed_pollution/Agricultural Area By Low, Medium and High Input Farms.csv'
name_2='USEFUL DATASETS/Meshed_pollution/Soil Erosion.csv'
name_dataset = "FINAL DATASET.csv"

In [6]:
#reading the first dataset to mesh

data_1 = pd.read_csv(name_1, encoding = "ISO-8859-1")
data_1.head(15)

,TIME,GEO_NAME,GEO,INDIC_AG,HECTARES
0,2010,AT,Austria,High-input farms,"963,256"
1,2010,AT,Austria,Low-input farms,"960,426"
2,2010,AT,Austria,Medium-input farms,"966,845"
3,2010,AT1,Ostösterreich,High-input farms,"368,358"
4,2010,AT1,Ostösterreich,Low-input farms,"276,171"
5,2010,AT1,Ostösterreich,Medium-input farms,"538,077"
6,2010,AT11,Burgenland,High-input farms,"50,581"
7,2010,AT11,Burgenland,Low-input farms,"46,362"
8,2010,AT11,Burgenland,Medium-input farms,"71,240"
9,2010,AT12,Niederösterreich,High-input farms,"317,777"


In [7]:
#reading the second file to mesh

data_2 = pd.read_csv(name_2, encoding = "ISO-8859-1")
data_2.head(15)

,TIME,GEO_NAME,GEO,SE_LEVELS,SE_SECTOR,TONNES PER HECTARE
0,2010,AT,Austria,Total,Agricultural areas (excluding pastures),5.0
1,2010,AT,Austria,Moderate,Agricultural areas (excluding pastures),7.1
2,2010,AT,Austria,Severe,Agricultural areas (excluding pastures),18.6
3,2010,AT,Austria,Total,Pastures and natural grassland,10.2
4,2010,AT,Austria,Moderate,Pastures and natural grassland,7.4
5,2010,AT,Austria,Severe,Pastures and natural grassland,19.1
6,2010,AT11,Burgenland (AT),Total,Agricultural areas (excluding pastures),2.9
7,2010,AT11,Burgenland (AT),Moderate,Agricultural areas (excluding pastures),7.1
8,2010,AT11,Burgenland (AT),Severe,Agricultural areas (excluding pastures),14.4
9,2010,AT11,Burgenland (AT),Total,Pastures and natural grassland,1.3


In [12]:
#scanning columns names and making a list containing all the colums names without duplicates
key_1=data_1.columns
key_2=data_2.columns
key_to_write=[]
for key in key_1:
    if key not in key_to_write:
        key_to_write.append(key)
for key2 in key_2:
    if key2 not in key_to_write:
        key_to_write.append(key2)
print(key_to_write)

null_1= len(key_to_write)-len(key_2)
null_2= len(key_to_write)-len(key_1)




['TIME', 'GEO_NAME', 'GEO', 'INDIC_AG', 'HECTARES', 'SE_LEVELS', 'SE_SECTOR', 'TONNES PER HECTARE']
2 3


In [15]:
#creating list to populate rows in the dataset

df_1=pd.DataFrame(data_1)
df_2=pd.DataFrame(data_2)
rows_to_add = []
row_to_add=[]
time = ""
geo_name = ""

def convert(list):
    return tuple(list)

for index, row in df_1.iterrows():
    for index_1, row_1 in df_2.iterrows():
        if (row["TIME"]<row_1["TIME"]):
            for item in row:
                row_to_add.append(item)
            #this number of Null corresponds to columns in the second dataset that the first does not have
            c=0
            while (c<null_2):
                row_to_add.append("Null")
                c+=1
            convert(row_to_add)
            rows_to_add.append(row_to_add)
            row_to_add=[]
            time = row["TIME"]
            geo_name = row["GEO_NAME"]
            df_1.drop(index, inplace=True)
            break
        elif (row["TIME"]>row_1["TIME"]):
            for item in row_1[:3]:
                row_to_add.append(item)
            #this number of Null corresponds to columns in the first dataset that the second does not have
            c=0
            while (c<null_1):
                row_to_add.append("Null")
                c+=1
            for item in row_1[3:]:
                row_to_add.append(item) 
            convert(row_to_add)
            rows_to_add.append(row_to_add)
            row_to_add=[]
            time = row_1["TIME"]
            geo_name = row_1["GEO_NAME"]
            df_2.drop(index_1, inplace=True)
        elif (row["TIME"]==row_1["TIME"]) & (row["GEO_NAME"]==row_1["GEO_NAME"]):
            for item in row:
                row_to_add.append(item)
            for item in row_1[3:]:
                row_to_add.append(item)
            convert(row_to_add)
            rows_to_add.append(row_to_add)
            row_to_add=[]
            time = row["TIME"]
            geo_name = row["GEO_NAME"]
            df_2.drop(index_1, inplace=True)
            df_1.drop(index, inplace=True)
            index_1+=1
            break
        elif (row["TIME"]==time) & (row["GEO_NAME"]==geo_name):
            for item in row:
                row_to_add.append(item)
            #this number of Null corresponds to columns in the second dataset that the first does not have
            c=0
            while (c<null_2):
                row_to_add.append("Null")
                c+=1
            convert(row_to_add)
            rows_to_add.append(row_to_add)
            row_to_add=[]
            time = row["TIME"]
            geo_name = row["GEO_NAME"]
            df_1.drop(index, inplace=True)
            break
        elif (row_1["TIME"]==time) & (row_1["GEO_NAME"]==geo_name):
            for item in row_1[:3]:
                row_to_add.append(item)
            #this number of Null corresponds to columns in the first dataset that the second does not have
            c=0
            while (c<null_1):
                row_to_add.append("Null")
                c+=1
            for item in row_1[3:]:
                row_to_add.append(item) 
            convert(row_to_add)
            rows_to_add.append(row_to_add)
            row_to_add=[]
            time = row_1["TIME"]
            geo_name = row_1["GEO_NAME"]
            df_2.drop(index_1, inplace=True)
        elif (row["GEO_NAME"][:2]>row_1["GEO_NAME"][:2]) & (row["TIME"]==row_1["TIME"]) | (row["GEO_NAME"][:2]==row_1["GEO_NAME"][:2]) | (row["GEO_NAME"]>row_1["GEO_NAME"]):
            if (geo_name==""):
                geo_name=row["GEO_NAME"]
            if (geo_name!=row_1["GEO_NAME"]) | (row["GEO_NAME"]<row_1["GEO_NAME"]):
                for item in row_1[:3]:
                    row_to_add.append(item)
                #this number of Null corresponds to columns in the first dataset that the second does not have
                c=0
                while (c<null_1):
                    row_to_add.append("Null")
                    c+=1
                for item in row_1[3:]:
                    row_to_add.append(item) 
                convert(row_to_add)
                rows_to_add.append(row_to_add)
                row_to_add=[]
                time = row_1["TIME"]
                geo_name = row_1["GEO_NAME"]
                df_2.drop(index_1, inplace=True)
            else:
                for item in row:
                    row_to_add.append(item)
                #this number of Null corresponds to columns in the second dataset that the first does not have
                c=0
                while (c<null_2):
                    row_to_add.append("Null")
                    c+=1
                convert(row_to_add)
                rows_to_add.append(row_to_add)
                row_to_add=[]
                time = row["TIME"]
                geo_name = row["GEO_NAME"]
                df_1.drop(index, inplace=True)
                break
        elif (row["GEO_NAME"][:2]<=row_1["GEO_NAME"][:2]) & (row["TIME"]==row_1["TIME"]) | (row["GEO_NAME"][:2]==row_1["GEO_NAME"][:2]) | (row["GEO_NAME"]<row_1["GEO_NAME"]):
            if (geo_name==row["GEO_NAME"]) | (row["GEO_NAME"]>row_1["GEO_NAME"]):
                for item in row_1[:3]:
                    row_to_add.append(item)
                #this number of Null corresponds to columns in the first dataset that the second does not have
                c=0
                while (c<null_1):
                    row_to_add.append("Null")
                    c+=1
                for item in row_1[3:]:
                    row_to_add.append(item) 
                convert(row_to_add)
                rows_to_add.append(row_to_add)
                row_to_add=[]
                time = row_1["TIME"]
                geo_name = row_1["GEO_NAME"]
                df_2.drop(index_1, inplace=True)
            else:
                for item in row:
                    row_to_add.append(item)
                #this number of Null corresponds to columns in the second dataset that the first does not have
                c=0
                while (c<null_2):
                    row_to_add.append("Null")
                    c+=1
                convert(row_to_add)
                rows_to_add.append(row_to_add)
                row_to_add=[]
                time = row["TIME"]
                geo_name = row["GEO_NAME"]
                df_1.drop(index, inplace=True)
                break

if len(df_1.index)>0: 
    for index_3, row_3 in df_1.iterrows():
        for item in row_3:
            row_to_add.append(item)
       #this number of Null corresponds to columns in the second dataset that the first does not have
        c=0
        while (c<null_2):
            row_to_add.append("Null")
            c+=1
        convert(row_to_add)
        rows_to_add.append(row_to_add)
        row_to_add=[]
if len(df_2.index)>0: 
    for index_4, row_4 in df_2.iterrows():
        print (index_4)
        for item in row_4[:3]:
            row_to_add.append(item)
       #this number of Null corresponds to columns in the first dataset that the second does not have
        c=0
        while (c<null_1):
            row_to_add.append("Null")
            c+=1
        for item in row_4[3:]:
            row_to_add.append(item) 
        convert(row_to_add)
        rows_to_add.append(row_to_add)
        row_to_add=[]


            


In [93]:

#populating new dataset

with open(name_dataset, encoding= "ISO-8859-1", mode='w') as my_file:
    my_writer = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    # write the column names
    my_writer.writerow(key_to_write)
    
    # access the rows of the query results
    for item in rows_to_add:
        # write in the csv
        my_writer.writerow(item)
